In [12]:
import sys
sys.path.append('/home/sunxin/project/AirGeoNet')
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from pathlib import Path

import numpy as np
from tqdm.auto import tqdm

from maploc.osm.viz import GeoPlotter
from maploc.utils.geo import BoundaryBox, Projection
from maploc.osm.tiling import TileManager

from pyproj import Proj, transform
import csv

import cv2 as cv
import torch

from torchvision import transforms as tvf

from PIL import Image

from typing import Literal
import glob


import natsort
import einops as ein
import pickle
from DINOV2.utilities import VLAD,DinoV2ExtractFeatures

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

class LocalArgs:
    
    # Input directory containing images
    in_dir: str = "/home/sunxin/project/AirGeoNet/datasets/VPAir/VPAir/queries"
    # Image file extension
    imgs_ext: str = "png"
    # Output directory where global descriptors will be stored
    out_dir: str = "/home/sunxin/project/AirGeoNet/datasets/VPAir/VPAir/VLAD_DataBase"
    # gps information 
    gps_path = '/home/sunxin/project/AirGeoNet/datasets/VPAir/VPAir/poses.csv'
    # c_center save path
    VLAD_path = '/home/sunxin/project/AirGeoNet/datasets/VPAir/VPAir'
    # Maximum edge length (expected) across all images
    max_img_size: int = 1024
 
    # Number of clusters (cluster centers for VLAD) - read from cache
    num_c: int = 64

    desc_layer: int = 31
    desc_facet: Literal["query", "key", "value", "token"] = "value"

    # Domain for use case (deployment environment)
    domain = "vpair"
    # Maximum image dimension
    max_img_size: int = 1024

    device = 'cuda'  


In [13]:
def parse_gps_file(path, projection: Projection = None):
    all_latlon = []

    with open(path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            all_latlon.append([float(row['lat']),float(row['lon'])])
    
    return np.array(all_latlon)



def prepare_VLAD(
    largs
):
    # Realpath expansion
    _ex = lambda x: os.path.realpath(os.path.expanduser(x))
    # Dino_v2 properties (parameters)

    save_dir = _ex(largs.out_dir)
    device = torch.device(largs.device)
    
    desc_layer: int = largs.desc_layer
    desc_facet: Literal["query", "key", "value", "token"] = largs.desc_facet
    num_c: int = largs.num_c
    domain:str =largs.domain
    max_img_size: int = largs.max_img_size
      
    # Ensure inputs are fine
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
        print(f"Creating directory: {save_dir}")
    else:
        print("Save directory already exists, overwriting possible!")

    # Load the DINO extractor model
    extractor = DinoV2ExtractFeatures("dinov2_vitg14", desc_layer,
        desc_facet, device=device)
    base_tf = tvf.Compose([ # Base image transformations
        tvf.ToTensor(),
        tvf.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
    ])

    imgs_dir = _ex(largs.in_dir)
    assert os.path.isdir(imgs_dir), "Input directory doesn't exist!"
    img_fnames = glob.glob(f"{imgs_dir}/*.{largs.imgs_ext}")
    img_fnames = natsort.natsorted(img_fnames)

    imgs_dir = _ex(largs.in_dir)
    assert os.path.isdir(imgs_dir), "Input directory doesn't exist!"
    img_fnames = glob.glob(f"{imgs_dir}/*.{largs.imgs_ext}")
    img_fnames = natsort.natsorted(img_fnames)
    
    img_patch_descs = []
    
    for img_fname in tqdm(img_fnames):
        with torch.no_grad():
            pil_img = Image.open(img_fname).convert('RGB')
            img_pt = base_tf(pil_img).to(device)
            if max(img_pt.shape[-2:]) > max_img_size:
                pass
            c,h,w = img_pt.shape
            h_new, w_new = (h // 14) * 14, (w // 14) * 14
            img_pt = tvf.CenterCrop((h_new,w_new))(img_pt)[None,...]
            ret = extractor(img_pt)
            img_patch_descs.append(ret.to('cpu'))
            

    result_tensor = torch.cat(img_patch_descs, dim=0)
    
    vlad = VLAD(num_c, desc_dim=result_tensor[0].shape[1], cache_dir= _ex(largs.VLAD_path))
    vlad.fit(ein.rearrange(result_tensor, "n k d -> (n k) d"))
    
    all_latlon = parse_gps_file(largs.gps_path)
    vlad_data = []
    for img_fname, ret, latlon in tqdm(zip(img_fnames, img_patch_descs, all_latlon), total=len(img_fnames)):

        # VLAD global descriptor
        gd = vlad.generate(ret.squeeze()) # VLAD:  [agg_dim]
        gd_np = gd.numpy()[np.newaxis, ...] # shape: [1, agg_dim]
        vlad_data.append({'gd_np':gd_np,'latlon':latlon})

    with open(f"{save_dir}/vlad_descriptors_64.pkl", 'wb') as file:
        pickle.dump(vlad_data, file)
        

In [14]:
prepare_VLAD(LocalArgs)

Save directory already exists, overwriting possible!


Using cache found in /home/sunxin/.cache/torch/hub/facebookresearch_dinov2_main


  0%|          | 0/2706 [00:00<?, ?it/s]

Caching cluster centers


  0%|          | 0/2706 [00:00<?, ?it/s]